# Selección features

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pprint import pprint

import warnings
warnings.filterwarnings('ignore')
pd.option_context('mode.use_inf_as_na', True)

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRFRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

import joblib
import os


from EDA_function_tools import *
from toolbox_DS import *

### Selección de features con ML

Parto de todas las features, convirtiendo las categóricas a numéricas

In [2]:
features_visual= ['Company', 'TypeName', 'OpSys', 'Resolution', 'Pantalla', 'Brand', 'Family', 'CPU_Model_disc', 'Gpu Brand', 'Gpu Type', 'GPU_Model_disc', 'Storage Type','Ram', 'Weight', 'Clock Speed (GHz)']

In [4]:
df = pd.read_csv('./data/df_precios_ordenadores.csv', index_col=0)
df

,Company,TypeName,Inches,Ram,OpSys,Weight,Price_euros,Resolution,Pantalla,Brand,Family,Model,Clock Speed (GHz),Gpu Brand,Gpu Model,Gpu Type,Storage Type
id,,,,,,,,,,,,,,,,,
268,Dell,Notebook,15.6,8,Windows 10,2.36,749.00,1366x768,Full HD,Intel,Core i7,7500U,2.7,AMD,Radeon R7 M445,Dedicated,HDD
347,Asus,Notebook,15.6,4,Windows 10,2.00,449.00,1366x768,Full HD,Intel,Pentium,Quad,1.1,Intel,HD Graphics 505,Unknown,HDD
353,Toshiba,Notebook,13.3,8,Windows 7,1.20,1460.00,1920x1080,Full HD,Intel,Core i5,6200U,2.3,Intel,HD Graphics 520,Unknown,SSD
578,Dell,Gaming,15.6,16,Windows 10,4.42,2868.99,3840x2160,4K Ultra HD,Intel,Core i7,7700HQ,2.8,Nvidia,GeForce GTX 1070,Dedicated,SSD
42,Dell,Ultrabook,12.5,8,Windows 7,1.26,1713.37,1920x1080,Full HD,Intel,Core i5,6300U,2.4,Intel,HD Graphics 520,Unknown,SSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Acer,2 in 1 Convertible,11.6,4,Chrome OS,1.25,389.00,1366x768,Touchscreen,Intel,Celeron,Dual,1.6,Intel,HD Graphics 400,Unknown,Flash Storage
716,Toshiba,Notebook,13.3,4,Windows 10,1.20,1195.00,1920x1080,Full HD,Intel,Core i5,6200U,2.3,Intel,HD Graphics 520,Unknown,SSD
517,Acer,Notebook,15.6,8,Windows 10,2.40,449.00,1366x768,Full HD,AMD,A8-Series,7410,2.2,AMD,Radeon R5,Dedicated,HDD


In [7]:
features_cat_engin = df.select_dtypes(include=(['object'])).columns.tolist()
features_cat_engin

['Company',
 'TypeName',
 'OpSys',
 'Resolution',
 'Pantalla',
 'Brand',
 'Family',
 'Model',
 'Gpu Brand',
 'Gpu Model',
 'Gpu Type',
 'Storage Type']

Tratamiento de features para todas las variables categóricas

In [8]:
df_engin = pd.get_dummies(df,columns=features_cat_engin, dtype=int)
df_engin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Columns: 251 entries, Inches to Storage Type_SSD
dtypes: float64(4), int32(246), int64(1)
memory usage: 919.1 KB


In [9]:
df_engin.columns

Index(['Inches', 'Ram', 'Weight', 'Price_euros', 'Clock Speed (GHz)',
       'Company_Acer', 'Company_Apple', 'Company_Asus', 'Company_Chuwi',
       'Company_Dell',
       ...
       'Gpu Model_Radeon RX 550', 'Gpu Model_Radeon RX 560',
       'Gpu Model_Radeon RX 580', 'Gpu Model_UHD Graphics 620',
       'Gpu Type_Dedicated', 'Gpu Type_Unknown', 'Storage Type_Flash Storage',
       'Storage Type_HDD', 'Storage Type_Hybrid', 'Storage Type_SSD'],
      dtype='object', length=251)

In [10]:
df_engin.shape

(912, 251)

In [34]:
df_engin.replace('<','')

,Inches,Ram,Weight,Price_euros,Clock Speed (GHz),Company_Acer,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,...,Gpu Model_Radeon RX 550,Gpu Model_Radeon RX 560,Gpu Model_Radeon RX 580,Gpu Model_UHD Graphics 620,Gpu Type_Dedicated,Gpu Type_Unknown,Storage Type_Flash Storage,Storage Type_HDD,Storage Type_Hybrid,Storage Type_SSD
id,,,,,,,,,,,,,,,,,,,,,
268,15.6,8,2.36,749.00,2.7,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
347,15.6,4,2.00,449.00,1.1,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
353,13.3,8,1.20,1460.00,2.3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
578,15.6,16,4.42,2868.99,2.8,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
42,12.5,8,1.26,1713.37,2.4,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,11.6,4,1.25,389.00,1.6,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
716,13.3,4,1.20,1195.00,2.3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
517,15.6,8,2.40,449.00,2.2,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [35]:


X_train_engin=df_engin.drop(columns='Price_euros')
y_train_engin = df['Price_euros']
print('X_train_engin', X_train_engin.shape)
print('y_train',y_train_engin.shape)

X_train_engin (912, 250)
y_train (912,)


In [36]:
X_train_engin.columns

Index(['Inches', 'Ram', 'Weight', 'Clock Speed (GHz)', 'Company_Acer',
       'Company_Apple', 'Company_Asus', 'Company_Chuwi', 'Company_Dell',
       'Company_Fujitsu',
       ...
       'Gpu Model_Radeon RX 550', 'Gpu Model_Radeon RX 560',
       'Gpu Model_Radeon RX 580', 'Gpu Model_UHD Graphics 620',
       'Gpu Type_Dedicated', 'Gpu Type_Unknown', 'Storage Type_Flash Storage',
       'Storage Type_HDD', 'Storage Type_Hybrid', 'Storage Type_SSD'],
      dtype='object', length=250)

____

ESta parte la dejo con los valores anteriores de X_train_engine y hago una nueva abajo para los valores de X_train_dummies

#### Selección de features numéricas mediante SelectKBeest y Anova

In [13]:
selector =SelectKBest(f_regression, k = 30)

x_data_kbest = selector.fit_transform(X_train_engin, y_train_engin)

X_train_kBest = pd.DataFrame(x_data_kbest, columns = selector.get_feature_names_out())
X_train_kBest.columns

Index(['Ram', 'Weight', 'Clock Speed (GHz)', 'Company_Razer',
       'TypeName_Gaming', 'TypeName_Notebook', 'TypeName_Ultrabook',
       'TypeName_Workstation', 'Resolution_1366x768', 'Resolution_3840x2160',
       'Pantalla_4K Ultra HD', 'Pantalla_Full HD', 'Family_Celeron',
       'Family_Core i3', 'Family_Core i7', 'Family_Xeon', 'Model_6006U',
       'Model_7700HQ', 'Model_7820HK', 'Model_Dual', 'Model_E3-1535M',
       'Gpu Brand_Intel', 'Gpu Brand_Nvidia', 'Gpu Model_GeForce GTX 1070',
       'Gpu Model_GeForce GTX 1080', 'Gpu Model_HD Graphics 400',
       'Gpu Type_Dedicated', 'Gpu Type_Unknown', 'Storage Type_HDD',
       'Storage Type_SSD'],
      dtype='object')

In [14]:
features_anova = X_train_kBest.columns

#### Selección usando modelo SelectFromModel

In [15]:
rf_selector = RandomForestRegressor(max_depth=5,random_state=42)
rf_selector.fit(X_train_engin, y_train_engin)

RandomForestRegressor(max_depth=5, random_state=42)

In [16]:
rf_selector.feature_importances_[rf_selector.feature_importances_ > rf_selector.feature_importances_.mean()]

array([0.00437707, 0.64205686, 0.06802458, 0.0358411 , 0.01326782,
       0.073668  , 0.02231673, 0.00468884, 0.01161548, 0.01583891,
       0.01811612, 0.00436629, 0.0043418 , 0.01258321, 0.0075665 ,
       0.0072811 , 0.00425102, 0.00675014, 0.0062376 ])

In [29]:
# Instanciamos el modelo
selector_model = SelectFromModel(estimator=rf_selector,threshold= 'mean')

# Entrenamos
selector_model.fit(X_train_engin,y_train_engin)

SelectFromModel(estimator=RandomForestRegressor(max_depth=5, random_state=42),
                threshold='mean')

In [30]:
selector_model.get_feature_names_out()

array(['Inches', 'Ram', 'Weight', 'Clock Speed (GHz)', 'Company_Razer',
       'TypeName_Notebook', 'TypeName_Workstation', 'OpSys_Windows 7',
       'Resolution_1920x1080', 'Family_Core i5', 'Family_Core i7',
       'Family_Xeon', 'Model_E3-1535M', 'Gpu Model_GeForce GTX 1070',
       'Gpu Model_GeForce GTX 1080', 'Gpu Model_Quadro M3000M',
       'Gpu Model_Quadro M620M', 'Storage Type_HDD', 'Storage Type_SSD'],
      dtype=object)

In [19]:
features_selector = selector_model.get_feature_names_out()

#### Selección usando RFE

In [20]:
# Instnacio modelo de clasificación
rf_RFE = RandomForestRegressor(max_depth=5,random_state=42)
# Sobre el modelo instancio el rfe
rfe = RFE(
    estimator=rf_RFE,
    n_features_to_select=30,
    step=1
)

# Entreno el modelo rfe
rfe.fit(X_train_engin,y_train_engin)
print(rfe.ranking_)
pd.DataFrame(rfe.ranking_,columns=['ranking'], index=X_train_engin.columns).sort_values('ranking')[:30]

[  1   1   1   1  70  62   1  74   1  83  78   1  71  76  31  53  82  73
   1  84  36  98 104   4  10 103   1   6   1 109 100  52 116  21  15 113
   1 117   1  38 119   1 120 106 140 122 148   3 163 105   7   1   1   1
  75 147  39  24  11 170 168 149 107 150 154   2  57  14   1   1  94  97
  37 142   1 165 162 205 173 137 164 185 188 146 143  89 123 171  47 174
 176  40  46  25  20 159  27   9 139 182 186 187   1  42 198 196  12  44
 207 208 210  63 214  35 177   1   1 160 199 201 211 215  26 218 221 220
 206 204 115  65  90 102  69  56  43  80 130 161  51  68  93  64   1 128
 124  48 144 129 193  92 108  54  91   1  67  34  28  58 202 127 189 126
 136 112 125  13 135   5  17 138  79   1  16   1 110 131 118 101  19  29
 167  30 141 175 158 132 155 151 121 133  49  60  50  32 153 145 172  45
 166  85 179  86  95  87  96 134 111  22 157  33   1 169  23 178   1 181
 180 183  61 190 192 194 191 195 197 203  55 209 212 216 217 213 219 200
  59 156 152   8 184  77  99 114  81  41  66  18  7

,ranking
Inches,1
OpSys_Windows 7,1
Resolution_1366x768,1
Resolution_1920x1080,1
Resolution_3840x2160,1
Pantalla_4K Ultra HD,1
Pantalla_Full HD,1
Family_Core i5,1
Family_Core i7,1
Family_Xeon,1


In [21]:
temp = pd.DataFrame(rfe.ranking_,columns=['ranking'], index=X_train_engin.columns).sort_values('ranking')[:30]
temp.index[:30]
features_rfe = temp.index[:30]

In [32]:
features_rfe

Index(['Inches', 'OpSys_Windows 7', 'Resolution_1366x768',
       'Resolution_1920x1080', 'Resolution_3840x2160', 'Pantalla_4K Ultra HD',
       'Pantalla_Full HD', 'Family_Core i5', 'Family_Core i7', 'Family_Xeon',
       'Model_6Y75', 'Model_7700HQ', 'Model_7820HK', 'Model_E3-1535M',
       'Gpu Brand_AMD', 'Gpu Model_GeForce GTX 1070',
       'Gpu Model_GeForce GTX 1080', 'Gpu Model_Quadro M3000M',
       'Gpu Model_Quadro M620M', 'Storage Type_HDD', 'TypeName_Workstation',
       'TypeName_Notebook', 'Storage Type_SSD', 'Company_Asus', 'Company_Dell',
       'Clock Speed (GHz)', 'Ram', 'Weight', 'Company_Razer', 'Company_HP'],
      dtype='object')

#### Selección por SFS

In [22]:
# Instancio modelo clasificación
rf_sfs = RandomForestRegressor(max_depth=5,random_state=42)
# Instancio el modelo de la features selection
sfs_fordward = SequentialFeatureSelector(
    rf_sfs,
    n_features_to_select=30,
    cv = 5,
    scoring='neg_median_absolute_error' 
)

# Entreno el modelo de features_selection
sfs_fordward.fit(X_train_engin,y_train_engin)

SequentialFeatureSelector(estimator=RandomForestRegressor(max_depth=5,
                                                          random_state=42),
                          n_features_to_select=30,
                          scoring='neg_median_absolute_error')

In [23]:
# printar las features seleccionadas
print("Elegidasd SFS:", sfs_fordward.get_feature_names_out())

Elegidasd SFS: ['Inches' 'Ram' 'Weight' 'Clock Speed (GHz)' 'Company_HP' 'OpSys_No OS'
 'OpSys_macOS' 'Pantalla_Full HD' 'Pantalla_Touchscreen' 'Family_Core i5'
 'Family_Core i7' 'Model_2.7GHz' 'Model_6100U' 'Model_6300U'
 'Model_6820HQ' 'Model_6Y75' 'Model_7700HQ' 'Model_7Y57' 'Model_8650U'
 'Model_9700P' 'Model_Dual' 'Gpu Model_FirePro W6150M'
 'Gpu Model_GeForce 960M' 'Gpu Model_GeForce GTX 1050 Ti'
 'Gpu Model_GeForce GTX 1060' 'Gpu Model_Iris Plus Graphics 640'
 'Gpu Model_Quadro M2000M' 'Gpu Model_Radeon R5 M420' 'Gpu Type_Unknown'
 'Storage Type_Flash Storage']


In [24]:
features_sfs = sfs_fordward.get_feature_names_out()

Instancio modelos

In [25]:
lin_reg = LinearRegression()
rf_model = RandomForestRegressor(max_depth=5, random_state=42)
gb= GradientBoostingRegressor(max_depth=5, random_state=42)
xgb = XGBRFRegressor(max_depth = 5, random_state = 42)
cat = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, random_state=42,verbose=0)

In [39]:
features_anova = ['Ram', 'Weight', 'Clock Speed (GHz)', 'Company_Razer',
       'TypeName_Gaming', 'TypeName_Notebook', 'TypeName_Ultrabook',
       'TypeName_Workstation', 'Resolution_1366x768', 'Resolution_3840x2160',
       'Pantalla_4K Ultra HD', 'Pantalla_Full HD', 'Family_Celeron',
       'Family_Core i3', 'Family_Core i7', 'Family_Xeon', 'Model_6006U',
       'Model_7700HQ', 'Model_7820HK', 'Model_Dual', 'Model_E3-1535M',
       'Gpu Brand_Intel', 'Gpu Brand_Nvidia', 'Gpu Model_GeForce GTX 1070',
       'Gpu Model_GeForce GTX 1080', 'Gpu Model_HD Graphics 400',
       'Gpu Type_Dedicated', 'Gpu Type_Unknown', 'Storage Type_HDD',
       'Storage Type_SSD']

features_selector = ['Inches', 'Ram', 'Weight', 'Clock Speed (GHz)', 'Company_Razer',
       'TypeName_Notebook', 'TypeName_Workstation', 'OpSys_Windows 7',
       'Resolution_1920x1080', 'Family_Core i5', 'Family_Core i7',
       'Family_Xeon', 'Model_E3-1535M', 'Gpu Model_GeForce GTX 1070',
       'Gpu Model_GeForce GTX 1080', 'Gpu Model_Quadro M3000M',
       'Gpu Model_Quadro M620M', 'Storage Type_HDD', 'Storage Type_SSD']


features_rfe = ['Inches', 'OpSys_Windows 7', 'Resolution_1366x768',
       'Resolution_1920x1080', 'Resolution_3840x2160', 'Pantalla_4K Ultra HD',
       'Pantalla_Full HD', 'Family_Core i5', 'Family_Core i7', 'Family_Xeon',
       'Model_6Y75', 'Model_7700HQ', 'Model_7820HK', 'Model_E3-1535M',
       'Gpu Brand_AMD', 'Gpu Model_GeForce GTX 1070',
       'Gpu Model_GeForce GTX 1080', 'Gpu Model_Quadro M3000M',
       'Gpu Model_Quadro M620M', 'Storage Type_HDD', 'TypeName_Workstation',
       'TypeName_Notebook', 'Storage Type_SSD', 'Company_Asus', 'Company_Dell',
       'Clock Speed (GHz)', 'Ram', 'Weight', 'Company_Razer', 'Company_HP']

features_sfs = ['Inches', 'Ram', 'Weight', 'Clock Speed (GHz)', 'Company_HP', 'OpSys_No OS',
 'OpSys_macOS', 'Pantalla_Full HD', 'Pantalla_Touchscreen', 'Family_Core i5',
 'Family_Core i7', 'Model_2.7GHz', 'Model_6100U', 'Model_6300U',
 'Model_6820HQ', 'Model_6Y75', 'Model_7700HQ', 'Model_7Y57', 'Model_8650U',
 'Model_9700P', 'Model_Dual', 'Gpu Model_FirePro W6150M',
 'Gpu Model_GeForce 960M', 'Gpu Model_GeForce GTX 1050 Ti',
 'Gpu Model_GeForce GTX 1060', 'Gpu Model_Iris Plus Graphics 640',
 'Gpu Model_Quadro M2000M', 'Gpu Model_Radeon R5 M420', 'Gpu Type_Unknown',
 'Storage Type_Flash Storage']


#### Prueba mejores modelos con cross_val_score

In [70]:
# Lista de features
features_list = [features_anova, features_selector,features_rfe,features_sfs]
features_list_name = ['ANNOVA', 'SelectFromModel','RFE','SFS']

# Nombre de los modelos:
model_name = ['LinearRegression','RandomForestregression','GradientBoost','XGBoostRegressor','CatBoost']

# Variables con modelos:
model_list = [lin_reg,rf_model,gb,xgb,cat]

for i,lista in enumerate(features_list):
    for j,modelo in enumerate(model_list):
        score = np.mean(cross_val_score(modelo, X_train_engin[lista], y_train_engin, cv = 5, scoring = 'neg_median_absolute_error')).round(2)
        print(f'Lista: {features_list_name[i]} y modelo: {model_name[j]}: {score}')
    print()

Lista: ANNOVA y modelo: LinearRegression: -173.68
Lista: ANNOVA y modelo: RandomForestregression: -166.59
Lista: ANNOVA y modelo: GradientBoost: -133.19
Lista: ANNOVA y modelo: XGBoostRegressor: -162.23
Lista: ANNOVA y modelo: CatBoost: -150.74

Lista: SelectFromModel y modelo: LinearRegression: -181.92
Lista: SelectFromModel y modelo: RandomForestregression: -156.25
Lista: SelectFromModel y modelo: GradientBoost: -124.27
Lista: SelectFromModel y modelo: XGBoostRegressor: -150.31
Lista: SelectFromModel y modelo: CatBoost: -143.34

Lista: RFE y modelo: LinearRegression: -174.21
Lista: RFE y modelo: RandomForestregression: -155.86
Lista: RFE y modelo: GradientBoost: -122.64
Lista: RFE y modelo: XGBoostRegressor: -154.11
Lista: RFE y modelo: CatBoost: -138.82

Lista: SFS y modelo: LinearRegression: -195.42
Lista: SFS y modelo: RandomForestregression: -145.14
Lista: SFS y modelo: GradientBoost: -132.1
Lista: SFS y modelo: XGBoostRegressor: -156.5
Lista: SFS y modelo: CatBoost: -146.24



### Optimización hiperparámetros


Para GBR (escalado)

In [72]:
X_train_model = X_train_engin[features_selector]

param_grid_gb_reg = {'max_depth': [4,5,6]}

    #'n_estimators': [200,300,400],
    #'max_depth': [4,5,6],}
    #'learning_rate': [0.02,0.05,0.1],}
    #'subsample': [0.8,1,1.5]
 
    


gbr_grid_5 = GridSearchCV(estimator=gb,
                     param_grid=param_grid_gb_reg, 
                     scoring='neg_mean_absolute_error', 
                     cv=5, 
                     n_jobs=-1)

gbr_grid_5.fit(X_train_model, y_train_engin)

GridSearchCV(cv=5,
             estimator=GradientBoostingRegressor(max_depth=5, random_state=42),
             n_jobs=-1, param_grid={'max_depth': [4, 5, 6]},
             scoring='neg_mean_absolute_error')

In [73]:
best_model_gb_reg = gbr_grid_5.best_estimator_
best_params_gb_reg = gbr_grid_5.best_params_
best_score_gb_reg = gbr_grid_5.best_score_

print(f'Best Parameters for Regression: {best_params_gb_reg}')
print(f'Best Score for Regression: {best_score_gb_reg}')


Best Parameters for Regression: {'max_depth': 5}
Best Score for Regression: -201.70581296583856


In [66]:
y_predict = gbr_grid_5.predict(X_train_model)

mae = mean_absolute_error(y_train_engin, y_predict)
print('MAE',mae)

MAE 104.17351480495036


In [67]:
y_predict

array([ 633.91251527,  305.33051744, 1308.37794823, 2855.31923917,
       1698.71566987, 1223.67660211,  471.1290832 , 1170.86317737,
       1275.8076505 , 1457.10530583,  299.62949232, 1024.1944624 ,
        800.48262677,  681.92089618,  509.90795056,  961.71402736,
        672.58870272,  579.68997114, 4845.50067259,  903.40497439,
        516.45708951,  584.07810815, 1531.80432853, 1694.56812002,
       2722.49402302,  420.92130743,  579.99325206,  649.84152711,
        311.78172009, 1074.96542049,  843.3428995 , 1182.4534314 ,
        976.75333636,  417.23530512,  339.77385756,  552.87660482,
        346.05717174,  337.15341481,  465.7440961 , 1286.61940735,
       1041.69226392, 1089.19794961,  976.51601635, 1229.46832718,
        664.57813382, 1432.79215562,  449.87794917,  444.95349776,
       2029.26278234, 1084.11034556,  404.49539723, 1400.43991935,
        850.14312816,  326.07462841,  699.19434422, 1670.45564843,
        889.11572803, 1181.58058431,  221.64757574, 1881.94031

Para RandomForest

In [59]:
X_train_model = X_train_engin[features_rfe]

param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['absolute_error'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

rf_model_grid = GridSearchCV(estimator=rf_model,
							   param_grid=param_grid,
							   scoring='neg_median_absolute_error',
							   cv=5,
							   n_jobs=-1)

rf_model_grid.fit(X_train_model, y_train_engin)

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=5, random_state=42),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['absolute_error'],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10, 20],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_median_absolute_error')

In [60]:
best_model_clf = rf_model_grid.best_estimator_
best_params_clf = rf_model_grid.best_params_
best_score_clf = rf_model_grid.best_score_

print(f'Best Parameters for Classification: {best_params_clf}')
print(f'Best Score for Classification: {best_score_clf}')

Best Parameters for Classification: {'bootstrap': True, 'criterion': 'absolute_error', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Score for Classification: -114.27735166666665


In [61]:
y_predict = rf_model_grid.predict(X_train_model)

mae = mean_absolute_error(y_train_engin, y_predict)
print('MAE',mae)

MAE 88.61387759502932


In [64]:
y_predict

array([ 767.71231667,  336.82346667, 1379.87095   , 2863.51245   ,
       1664.17158333, 1192.66045   ,  455.7373    , 1180.91156667,
       1299.00583333, 1637.28416667,  338.54478333,  943.43376667,
        920.09796667,  611.02821667,  518.05993333, 1027.91208333,
        678.85453333,  578.95583333, 3735.40466667,  886.70801667,
        512.609     ,  607.9763    , 1442.42716667, 1536.07586667,
       2469.31196667,  423.57961667,  567.2771    ,  671.73866667,
        330.73648333, 1000.39951667,  779.86743333, 1201.23523333,
        964.62138333,  497.57316667,  352.64813333,  570.6734    ,
        355.99428333,  366.05235   ,  432.29133333, 1200.53271667,
       1062.02485   , 1096.87568333,  848.46236667, 1326.46068333,
        636.60683333, 1235.31781667,  447.7912    ,  476.22136667,
       1937.19873333,  983.68218333,  410.22063333, 1384.54825   ,
        806.00533333,  372.99248333,  629.99115   , 1651.15983333,
        978.42466667, 1358.83136667,  252.76058333, 1861.13271

Para CatBoost


In [71]:
X_train_model = X_train_engin[features_anova]

param_grid = {
    'depth': [6, 7,8],
    'learning_rate': [0.03,0.05,0.06],
    'iterations': [1300,1400,1500]
}

# Instanciamos GridSearchCV
cat_grid = GridSearchCV(estimator=cat,
                           param_grid=param_grid,
                           scoring='neg_mean_absolute_error',
                           cv=5,
                           verbose=0)

# Ejecutamos el ajuste de hiperparámetros
cat_grid.fit(X_train_model, y_train_engin)

# Imprimimos los mejores parámetros encontrados
print(f'Mejores hiperparámetros: {cat_grid.best_params_}')
print(f'Mejor MAE: {cat_grid.best_score_}')

Mejores hiperparámetros: {'depth': 6, 'iterations': 1500, 'learning_rate': 0.03}
Mejor MAE: -201.30899749357613


In [ ]:
import joblib
import os

model_path = os.path.join('models', 'rf_model_grid.pkl')
joblib.dump(rf_model_grid, model_path)

['models\\rf_model_grid.pkl']

In [ ]:
model_path = os.path.join('models', 'knn_hip_grid.pkl')
joblib.dump(knn_hip_grid, model_path)

['models\\knn_hip_grid.pkl']

______

ESta parte es la misma que la de arriba pero con X_train_dummies

#### Selección de features numéricas mediante SelectKBeest y Anova

In [ ]:
selector =SelectKBest(f_regression, k = 30)

x_data_kbest = selector.fit_transform(X_train_dummies, y_train_engin)

X_train_kBest = pd.DataFrame(x_data_kbest, columns = selector.get_feature_names_out())
X_train_kBest.columns

Index(['Storage Type_SSD', 'Company_Acer', 'TypeName_Workstation',
       'TypeName_Notebook', 'Family_Core i3', 'Resolution_1920x1080',
       'Resolution_1366x768', 'CPU_Model_disc_Baja', 'TypeName_Ultrabook',
       'GPU_Model_disc_Alta', 'Pantalla_4K Ultra HD', 'Storage Type_HDD',
       'Pantalla_Full HD', 'Family_Celeron', 'Resolution_2560x1440',
       'GPU_Model_disc_Baja', 'Gpu Type_Unknown', 'Gpu Brand_AMD',
       'Gpu Brand_Intel', 'Storage Type_Flash Storage', 'Resolution_3840x2160',
       'Clock Speed (GHz)', 'Family_Core i7', 'Weight', 'CPU_Model_disc_Alta',
       'Ram', 'Company_Razer', 'Gpu Brand_Nvidia', 'Gpu Type_Dedicated',
       'TypeName_Gaming'],
      dtype='object')

In [ ]:
features_anova = X_train_kBest.columns

#### Selección usando modelo SelectFromModel

In [ ]:
rf_selector = RandomForestRegressor(max_depth=5,random_state=42)
rf_selector.fit(X_train_dummies, y_train_engin)

RandomForestRegressor(max_depth=5, random_state=42)

In [ ]:
rf_selector.feature_importances_[rf_selector.feature_importances_ > rf_selector.feature_importances_.mean()]

array([0.01110661, 0.0132201 , 0.04117771, 0.18511712, 0.01544202,
       0.02553624, 0.08586699, 0.50823607, 0.01945172])

In [ ]:
# Instanciamos el modelo
selector_model = SelectFromModel(estimator=rf_selector,threshold= 'median')

# Entrenamos
selector_model.fit(X_train_dummies,y_train_engin)

SelectFromModel(estimator=RandomForestRegressor(max_depth=5, random_state=42),
                threshold='median')

In [ ]:
selector_model.get_feature_names_out()

array(['Family_Core i5', 'Storage Type_SSD', 'OpSys_Windows 10',
       'TypeName_Workstation', 'TypeName_Notebook', 'Family_Core i3',
       'Company_Asus', 'Family_Core M', 'Resolution_1920x1080',
       'Resolution_1366x768', 'CPU_Model_disc_Baja', 'TypeName_Ultrabook',
       'Family_Pentium', 'GPU_Model_disc_Alta', 'Pantalla_Touchscreen',
       'Company_MSI', 'Storage Capacity (GB)', 'Pantalla_4K Ultra HD',
       'Company_Dell', 'Storage Type_HDD', 'Pantalla_Full HD',
       'Pantalla_Quad HD+', 'TypeName_2 in 1 Convertible', 'OpSys_No OS',
       'Family_Celeron', 'Resolution_2560x1440', 'Company_HP',
       'GPU_Model_disc_Media', 'Gpu Type_Unknown', 'Gpu Brand_AMD',
       'CPU_Model_disc_Media', 'Resolution_2880x1800', 'Gpu Brand_Intel',
       'Resolution_3840x2160', 'Resolution_3200x1800',
       'Clock Speed (GHz)', 'Inches', 'Family_Core i7', 'Company_Lenovo',
       'Weight', 'Company_Toshiba', 'Brand_Intel', 'CPU_Model_disc_Alta',
       'Ram', 'Company_Razer', 'Gpu Br

In [ ]:
features_selector = selector_model.get_feature_names_out()

#### Selección usando RFE

In [ ]:
# Instnacio modelo de clasificación
rf_RFE = RandomForestRegressor(max_depth=5,random_state=42)
# Sobre el modelo instancio el rfe
rfe = RFE(
    estimator=rf_RFE,
    n_features_to_select=30,
    step=1
)

# Entreno el modelo rfe
rfe.fit(X_train_dummies,y_train_engin)
print(rfe.ranking_)
pd.DataFrame(rfe.ranking_,columns=['ranking'], index=X_train_dummies.columns).sort_values('ranking')[:30]

[ 1 33  1  1 31 29 28 26 27 37 42  1  1 30  8 39 25 49 22 60  1 19 53  1
 11 34  1 44 46  1 55  1 18  1 12 65 16  1  1  1  1 47  1  6  2  3  1 67
 68 66  1  1 23 38  1  7  4 63 40  1 43 52 54 45 17 50  1 56 57 58 59 61
 13  1  1 64 62 35  1 48  5 51  1 10 15 36 32 14  1  1 24  9  1 41 20 21
  1]


,ranking
Family_Core i5,1
TypeName_Ultrabook,1
GPU_Model_disc_Alta,1
Storage Capacity (GB),1
Pantalla_4K Ultra HD,1
Company_Dell,1
Storage Type_HDD,1
Pantalla_Full HD,1
Family_Celeron,1
Resolution_2560x1440,1


In [ ]:
temp = pd.DataFrame(rfe.ranking_,columns=['ranking'], index=X_train_dummies.columns).sort_values('ranking')[:30]
temp.index[:30]
features_rfe = temp.index[:30]

#### Selección por SFS

In [ ]:
# Instancio modelo clasificación
rf_sfs = RandomForestRegressor(max_depth=5,random_state=42)
# Instancio el modelo de la features selection
sfs_fordward = SequentialFeatureSelector(
    rf_sfs,
    n_features_to_select=30,
    cv = 4,
    scoring='neg_median_absolute_error' 
)

# Entreno el modelo de features_selection
sfs_fordward.fit(X_train_dummies,y_train_engin)

SequentialFeatureSelector(cv=4,
                          estimator=RandomForestRegressor(max_depth=5,
                                                          random_state=42),
                          n_features_to_select=30,
                          scoring='neg_median_absolute_error')

In [ ]:
# printar las features seleccionadas
print("Elegidasd SFS:", sfs_fordward.get_feature_names_out())

Elegidasd SFS: ['Family_Core i5' 'OpSys_Windows 10 S' 'OpSys_Windows 10'
 'Family_A6-Series' 'TypeName_Notebook' 'Family_Core i3'
 'CPU_Model_disc_Desconocida' 'Family_Ryzen' 'Brand_AMD' 'Family_Cortex'
 'Company_Fujitsu' 'CPU_Model_disc_Baja' 'TypeName_Ultrabook'
 'GPU_Model_disc_Alta' 'Pantalla_Touchscreen' 'Pantalla_Retina Display'
 'Storage Capacity (GB)' 'Pantalla_Quad HD+' 'TypeName_2 in 1 Convertible'
 'TypeName_Netbook' 'Resolution_2880x1800' 'Clock Speed (GHz)' 'Inches'
 'OpSys_Android' 'Weight' 'Company_Toshiba' 'Brand_Intel'
 'Resolution_2400x1600' 'Company_Samsung' 'Ram']


In [ ]:
features_sfs = sfs_fordward.get_feature_names_out()

#### Prueba mejores modelos con cross_val_score

In [ ]:
# Lista de features
features_list = [features_visual_final, features_anova, features_selector,features_rfe,features_sfs]
features_list_name = ['Visual','ANNOVA', 'SelectFromModel','RFE','SFS']

# Nombre de los modelos:
model_name = ['LinearRegression','RandomForestregression','XGBoostRegressor','KNN']

# Variables con modelos:
model_list = [lin_reg,rf_model,xgb,knn]

for i,lista in enumerate(features_list):
    for j,modelo in enumerate(model_list):
        score = np.mean(cross_val_score(modelo, X_train_dummies[lista], y_train_engin, cv = 5, scoring = 'neg_mean_absolute_error')).round(2)
        print(f'Lista: {features_list_name[i]} y modelo: {model_name[j]}: {score}')
    print()

Lista: Visual y modelo: LinearRegression: -220.26
Lista: Visual y modelo: RandomForestregression: -220.66
Lista: Visual y modelo: XGBoostRegressor: -221.94
Lista: Visual y modelo: KNN: -209.5

Lista: ANNOVA y modelo: LinearRegression: -232.87
Lista: ANNOVA y modelo: RandomForestregression: -226.35
Lista: ANNOVA y modelo: XGBoostRegressor: -228.74
Lista: ANNOVA y modelo: KNN: -223.76

Lista: SelectFromModel y modelo: LinearRegression: -220.51
Lista: SelectFromModel y modelo: RandomForestregression: -218.9
Lista: SelectFromModel y modelo: XGBoostRegressor: -220.88
Lista: SelectFromModel y modelo: KNN: -229.57

Lista: RFE y modelo: LinearRegression: -227.25
Lista: RFE y modelo: RandomForestregression: -218.81
Lista: RFE y modelo: XGBoostRegressor: -221.96
Lista: RFE y modelo: KNN: -232.42

Lista: SFS y modelo: LinearRegression: -262.02
Lista: SFS y modelo: RandomForestregression: -227.34
Lista: SFS y modelo: XGBoostRegressor: -226.23
Lista: SFS y modelo: KNN: -245.53



### Optimización hiperparámetros


Para RandomForest

In [ ]:
X_train_model = X_train_dummies[features_visual_final]

param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['absolute_error'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

rf_model_grid = GridSearchCV(estimator=rf_model,
							   param_grid=param_grid,
							   scoring='neg_mean_absolute_error',
							   cv=5,
							   n_jobs=-1)

rf_model_grid.fit(X_train_model, y_train_engin)

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=5, random_state=42),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['absolute_error'],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10, 20],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_absolute_error')

In [ ]:
best_model_clf = rf_model_grid.best_estimator_
best_params_clf = rf_model_grid.best_params_
best_score_clf = rf_model_grid.best_score_

print(f'Best Parameters for Classification: {best_params_clf}')
print(f'Best Score for Classification: {best_score_clf}')

Best Parameters for Classification: {'bootstrap': False, 'criterion': 'absolute_error', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Score for Classification: -174.27753003182613


In [ ]:
y_predict = rf_model_grid.predict(X_train_model)

mae = mean_absolute_error(y_train_engin, y_predict)
print('MAE',mae)

MAE 37.05109117324569


In [ ]:
y_predict

array([ 750.695075,  375.965   , 1446.9485  , 2871.07995 , 1698.4318  ,
       1112.54555 ,  415.15475 , 1152.72275 , 1199.10995 , 1671.2595  ,
        310.1525  ,  867.77    , 1107.4556  ,  648.6991  ,  501.41    ,
       1103.37595 ,  630.075075,  556.59915 , 4853.5859  ,  856.442825,
        562.635   ,  655.98375 , 1385.672475, 1379.40835 , 2697.043125,
        389.68    ,  566.607975,  697.077375,  306.5125  ,  981.237175,
        736.6241  , 1145.2813  ,  894.8339  ,  610.554725,  358.2225  ,
        580.165725,  405.4976  ,  367.2725  ,  370.55505 , 1067.80245 ,
       1104.388725, 1096.0559  ,  723.62205 , 1414.115   ,  495.      ,
       1259.7795  ,  419.9576  ,  478.904725, 2139.0501  , 1024.7629  ,
        434.197875, 1548.86    ,  728.9125  ,  346.145   ,  654.6939  ,
       1708.2483  , 1093.745   , 1505.536075,  239.525   , 1921.86935 ,
        416.0075  ,  469.129725,  283.3175  , 1443.53195 , 1104.388725,
       1895.255   , 1111.055   ,  631.140025,  356.578325,  347.

Para KNN


In [ ]:
X_train_model = X_train_dummies[features_visual_final]

param_grid_knn_reg = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Instanciar el modelo KNN para regresión
knn_hip = KNeighborsRegressor()

# Realizar la búsqueda de hiperparámetros
knn_hip_grid = GridSearchCV(estimator=knn_reg,
                                param_grid=param_grid_knn_reg,
                                scoring='neg_mean_absolute_error',
                                cv=5,
                                n_jobs=-1)

knn_hip_grid.fit(X_train_model, y_train_engin)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [3, 5, 7, 9, 11],
                         'weights': ['uniform', 'distance']},
             scoring='neg_mean_absolute_error')

In [ ]:
# Obtener el mejor modelo y sus parámetros
best_model_knn_hip = knn_hip_grid.best_estimator_
best_params_knn_hip = knn_hip_grid.best_params_
best_score_knn_hip = knn_hip_grid.best_score_

In [ ]:
print(f'Best Parameters for Regression: {best_params_knn_hip}')
print(f'Best Score for Regression: {best_score_knn_hip}')

Best Parameters for Regression: {'algorithm': 'kd_tree', 'n_neighbors': 5, 'weights': 'distance'}
Best Score for Regression: -200.7940075140802


Para XGBoost

In [ ]:
X_train_model = X_train_dummies[features_sfs]


param_grid_xgb_reg = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

grid_search_xgb_reg = GridSearchCV(estimator=xgb,
                                    param_grid=param_grid_xgb_reg,
                                    scoring='neg_mean_absolute_error',
                                    cv=5,
                                    n_jobs=-1)

grid_search_xgb_reg.fit(X_train_model, y_train_engin)

GridSearchCV(cv=5,
             estimator=XGBRFRegressor(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric=None, feature_types=None,
                                      gamma=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=None,
                                      max_bin=None, max_cat_threshold=None,
                                      max_...
                                      n_jobs=None, num_parallel_tree=None,
                                      objective='reg:squarederror',
                                      random_state=42, reg_alpha=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1, 0.2],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 500],
                         'reg_alpha': [0, 0.01, 0.1], 'reg_lambda': [1, 1.5, 2],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_absolute_error')

In [ ]:
best_model_xgb_reg = grid_search_xgb_reg.best_estimator_
best_params_xgb_reg = grid_search_xgb_reg.best_params_
best_score_xgb_reg = grid_search_xgb_reg.best_score_

print(f'Best Parameters for Regression: {best_params_xgb_reg}')
print(f'Best Score for Regression: {best_score_xgb_reg}')

Best Parameters for Regression: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 1.0}
Best Score for Regression: -455.1659173329552


In [ ]:
import joblib
import os

model_path = os.path.join('models', 'rf_model_grid.pkl')
joblib.dump(rf_model_grid, model_path)

['models\\rf_model_grid.pkl']

### Comparo contra Test


In [ ]:
y_predict_test = rf_model_grid.predict(X_test_dummies)
y_predict_test

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Brand_Samsung
- CPU_Model_disc_Desconocida
- Family_A4-Series
- Family_Cortex
- Gpu Brand_ARM
- ...


In [ ]:
columnas_train = X_train_dummies.columns.to_list()
X_test_dummies = X_test_dummies[columnas_train]
y_predict_test = knn_hip_grid.predict(X_test_dummies)
y_predict_test

NameError: name 'X_train_dummies' is not defined